In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import newton
import os

# =================================================================
# 1. CARGA DE DATOS DESDE EL ARCHIVO CSV
# =================================================================

NOMBRE_ARCHIVO = 'Datos de Simulacion.csv'
GRADO_POLINOMIO = 4

# Verificar si el archivo existe antes de intentar leerlo
if not os.path.exists(NOMBRE_ARCHIVO):
    print(f"Error: No se encontró el archivo '{NOMBRE_ARCHIVO}'. Asegúrate de que esté en el directorio correcto.")
    exit()

try:
    df = pd.read_csv(NOMBRE_ARCHIVO)
    print(f"Datos cargados exitosamente desde '{NOMBRE_ARCHIVO}'. Total de filas: {len(df)}")
except Exception as e:
    print(f"Error al leer el archivo CSV: {e}")
    exit()

# =================================================================
# 2. DEFINICIÓN DE FUNCIONES NUMÉRICAS
# =================================================================

# Función de eficiencia (E = |CL| / CD)
def E(alpha, poly_CL, poly_CD):
    """Calcula la eficiencia aerodinámica usando los polinomios ajustados."""
    # Nota: CD_Drag debe ser siempre positivo en el rango de interés.
    return np.abs(poly_CL(alpha)) / poly_CD(alpha)

# Función Derivada de E (dE/d_alpha) - Raíz para Newton-Raphson
def derivada_E(alpha, poly_CL, poly_CD):
    """Calcula la derivada de la eficiencia (dE/d_alpha) usando la regla del cociente."""
    CL = poly_CL(alpha)
    CD = poly_CD(alpha)
    CL_prime = poly_CL.deriv()(alpha)
    CD_prime = poly_CD.deriv()(alpha)
    # Regla del cociente para E' = (CL * CD' - CL' * CD) / (CD^2)
    # Asumimos que la optimización se realiza en la rama donde CL es negativo (downforce)
    return (CL * CD_prime - CL_prime * CD) / (CD**2)

# =================================================================
# 3. PROCESO DE VALIDACIÓN E ITERACIÓN SOBRE CASOS DE PRUEBA
# =================================================================

print("="*80)
print("VALIDACIÓN Y PRUEBAS: OPTIMIZACIÓN DEL ÁNGULO DE ATAQUE ÓPTIMO")
print("="*80)

# Iterar sobre cada caso de prueba único usando la columna 'Caso_ID'
for case_id in df['Caso_ID'].unique():
    # 3.1 Extraer datos para el caso actual
    data = df[df['Caso_ID'] == case_id].copy()

    # Asegurarse de que los nombres de las columnas coincidan con el CSV
    configuracion = data['Configuracion'].iloc[0]
    alpha_data = data['Alpha_Grados'].values
    CL_data = data['CL_Downforce'].values
    CD_data = data['CD_Drag'].values
    Eficiencia_data = np.abs(CL_data / CD_data)

    # 3.2 Regresión Polinomial
    # El grado del polinomio se puede ajustar si el error es muy alto
    coef_CL = np.polyfit(alpha_data, CL_data, GRADO_POLINOMIO)
    poly_CL = np.poly1d(coef_CL)
    coef_CD = np.polyfit(alpha_data, CD_data, GRADO_POLINOMIO)
    poly_CD = np.poly1d(coef_CD)

    # 3.3 Función Wrapper para Newton (necesita pasar los polinomios)
    derivada_wrapper = lambda alpha: derivada_E(alpha, poly_CL, poly_CD)

    # 3.4 Estimación Inicial (Guess)
    alpha_guess = alpha_data[np.argmax(Eficiencia_data)]

    # 3.5 Aplicar Método de Newton-Raphson
    try:
        alpha_optimo = newton(
            func=derivada_wrapper,
            x0=alpha_guess,
            tol=1e-6, # Alta precisión para la validación
            maxiter=100
        )

        # 3.6 Calcular Resultados Finales
        CL_optimo = poly_CL(alpha_optimo)
        CD_optimo = poly_CD(alpha_optimo)
        Eficiencia_maxima = E(alpha_optimo, poly_CL, poly_CD)

        # 3.7 Cálculo del error (RMSE) para la validación del modelo
        CL_predicho = poly_CL(alpha_data)
        CD_predicho = poly_CD(alpha_data)
        rmse_CL = np.sqrt(np.mean((CL_data - CL_predicho)**2))
        rmse_CD = np.sqrt(np.mean((CD_data - CD_predicho)**2))

        # 3.8 Presentación Detallada de Resultados
        print(f"\n--- CASO {case_id}: {configuracion.upper()} ---")
        print(f"|  MÉTODO DE ENTRADA Y MODELADO:")
        print(f"|    -> Puntos de datos: {len(alpha_data)}")
        print(f"|    -> RMSE CL (Error de Regresión): {rmse_CL:.6f}")
        print(f"|    -> RMSE CD (Error de Regresión): {rmse_CD:.6f}")
        print(f"|    -> Estimación Inicial (α_guess): {alpha_guess:.2f} grados")
        print(f"|")
        print(f"|  RESULTADO DE OPTIMIZACIÓN (NEWTON-RAPHSON):")
        print(f"|    -> ÁNGULO ÓPTIMO (α_optimo): {alpha_optimo:.4f} grados")
        print(f"|    -> COEF. DOWNFORCE (CL): {CL_optimo:.4f}")
        print(f"|    -> COEF. DRAG (CD): {CD_optimo:.5f}")
        print(f"|    -> EFICIENCIA MÁXIMA (|CL/CD|): {Eficiencia_maxima:.3f}")

    except RuntimeError as e:
        print(f"\n--- CASO {case_id}: {configuracion.upper()} ---")
        print(f"|  FALLO: Newton-Raphson no convergió. Error: {e}")
        print(f"|  *Sugerencia: Revisar el grado del polinomio o la estimación inicial.*")

print("\n" + "="*80)
print("FIN DEL ANÁLISIS DE VALIDACIÓN")
print("="*80)

Datos cargados exitosamente desde 'Datos de Simulacion.csv'. Total de filas: 5000
VALIDACIÓN Y PRUEBAS: OPTIMIZACIÓN DEL ÁNGULO DE ATAQUE ÓPTIMO

--- CASO 1: MAXIMA_EFICIENCIA ---
|  MÉTODO DE ENTRADA Y MODELADO:
|    -> Puntos de datos: 1000
|    -> RMSE CL (Error de Regresión): 0.005014
|    -> RMSE CD (Error de Regresión): 0.001020
|    -> Estimación Inicial (α_guess): 0.66 grados
|
|  RESULTADO DE OPTIMIZACIÓN (NEWTON-RAPHSON):
|    -> ÁNGULO ÓPTIMO (α_optimo): 0.6967 grados
|    -> COEF. DOWNFORCE (CL): -0.7419
|    -> COEF. DRAG (CD): 0.02466
|    -> EFICIENCIA MÁXIMA (|CL/CD|): 30.084

--- CASO 2: ALTA_CARGA ---
|  MÉTODO DE ENTRADA Y MODELADO:
|    -> Puntos de datos: 1000
|    -> RMSE CL (Error de Regresión): 0.005158
|    -> RMSE CD (Error de Regresión): 0.000987
|    -> Estimación Inicial (α_guess): 1.08 grados
|
|  RESULTADO DE OPTIMIZACIÓN (NEWTON-RAPHSON):
|    -> ÁNGULO ÓPTIMO (α_optimo): 1.1286 grados
|    -> COEF. DOWNFORCE (CL): -1.0974
|    -> COEF. DRAG (CD): 0.0700